In [1]:
# Basic Imports
import numpy as np
from scipy.stats import uniform, invwishart, matrix_normal, norm
from scipy.stats import multivariate_normal as mvn
import pandas as pd
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import seaborn as sns
from time import localtime, strftime

# sklearn imports
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.linear_model import LogisticRegression as LR
from sklearn.naive_bayes import GaussianNB as GNB

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import OneHotEncoder

# Sarcoma Dataset

In [2]:
# Read in data
data = pd.read_csv("../data/combined_sarcoma_data.csv")

# Clean column names
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('-', '_')

# Change MALE to 0, FEMALE to 1
data = data.replace({"MALE": 0, "FEMALE":1})

# Drop any rows with missing data
data = data.dropna(axis=0)

# Normalize the data
data['Y'] = (data['Y'] - data['Y'].mean()) / data['Y'].std()

# Train-test split
X_train = data[~data["short_histo"].isin(['SS', 'MPNST'])]

X_test = data[data["short_histo"].isin(['SS', 'MPNST'])]
# X_test = data[data["short_histo"].isin(['SS'])]
# X_test = data[data["short_histo"].isin(['MPNST'])]

features = ['age_at_diagnosis', 'gender', 'JUN',
       'VGLL3', 'TERT', 'MAP3K5', 'UST', 'CDKN2A', 'YAP1', 'CDKN1B', 'PTPRQ',
       'RB1', 'TP53', 'MYOCD', 'NF1', 'CCNE1', 'CEBPA', 'ZNF552', 'ATRX',
       'PTEN', 'DDIT3', 'CDK4', 'HMGA2', 'MDM2', 'FRS2', 'Silent_per_Mb',
       'Non_silent_per_Mb', 'CD274', 'CTLA4', 'HAVCR2', 'LAG3', 'PDCD1', 
       'TCF7', 'TIGIT']

output = 'Y'

/var/folders/wm/2drf_bfd1kd_992mj3hlps740000gn/T/ipykernel_95350/1994202095.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({"MALE": 0, "FEMALE":1})


In [3]:
# Regular Tree

performances = []

for i in range(100):
    tree = DTR(random_state=i)
    tree.fit(X_train[features], X_train[output])
    pred = tree.predict(X_test[features])
    tree_mse = mse(X_test[output], pred)
    performances.append(tree_mse)

print("Tree MSE:", np.mean(performances))

Tree MSE: 1.7861912046397423


In [5]:
# Regular random forest

performances = []

for i in range(100):
    tree = RFR(n_estimators=5, random_state=i)
    tree.fit(X_train[features], X_train[output])
    pred = tree.predict(X_test[features])
    forest_mse = mse(X_test[output], pred)
    performances.append(forest_mse)

print("Forest (5 Trees) MSE:", np.mean(performances))

Forest (5 Trees) MSE: 0.8258487463393606


In [6]:
# Regular random forest

performances = []

for i in range(20):
    tree = RFR(n_estimators=100, random_state=i)
    tree.fit(X_train[features], X_train[output])
    pred = tree.predict(X_test[features])
    forest_mse = mse(X_test[output], pred)
    performances.append(forest_mse)

print("Forest (100 Trees) MSE:", np.mean(performances))

Forest (100 Trees) MSE: 0.6942883759127036


In [7]:
# Build group classifier
group_clf = LR()  # GNB()  # DTC or RFC or LR or something else?
group_clf.fit(X_train[features], X_train['short_histo'])
group_pred = group_clf.predict_proba(X_test[features])

print(group_clf.classes_)

group_pred = group_pred  # + 0.5 #

# Normalize group predictions
row_sums = group_pred.sum(axis=1)
group_pred = group_pred / row_sums[:, np.newaxis]

for test_group in ['SS', 'MPNST']:
    rows = np.where(X_test['short_histo'] == test_group)
    average = np.mean(group_pred[rows,], axis=1)
    group_pred[rows] = average

['DDLPS' 'MFS' 'STLMS' 'ULMS' 'UPS']


/opt/homebrew/anaconda3/envs/me_reg/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
tmp = group_clf.coef_
len(features)

34

In [47]:
# Mixture of Trees

performances = []

for i in range(100):

    list_of_trees = []
    for group in ['DDLPS', 'MFS', 'STLMS', 'ULMS', 'UPS']:
        tree = DTR(random_state=i)
        tree.fit(
            X_train[X_train["short_histo"] == group][
                features
            ],
            X_train[X_train["short_histo"] == group][output],
        )
        list_of_trees.append(tree)

    preds = np.array(
        [
            tree.predict(X_test[features])
            for tree in list_of_trees
        ]
    )
    preds = preds.T
    num = preds.shape[0]
    pred = [np.dot(preds[i, :], group_pred[i, :]) for i in range(num)]
    performances.append(mse(X_test[output], pred))



print("Mixture of Trees MSE:", np.mean(performances))

Mixture of Trees MSE: 0.6432881083999925


In [46]:
# Mixture of RF

performances = []

for i in range(100):

    list_of_trees = []
    for group in ['DDLPS', 'MFS', 'STLMS', 'ULMS', 'UPS']:
        tree = RFR(n_estimators=20, random_state=i)
        tree.fit(
            X_train[X_train["short_histo"] == group][
                features
            ],
            X_train[X_train["short_histo"] == group][output],
        )
        list_of_trees.append(tree)

    preds = np.array(
        [
            tree.predict(X_test[features])
            for tree in list_of_trees
        ]
    )
    preds = preds.T
    num = preds.shape[0]
    pred = [np.dot(preds[i, :], group_pred[i, :]) for i in range(num)]
    performances.append(mse(X_test[output], pred))



print("Mixture of 20 Tree Forest MSE:", np.mean(performances))

Mixture of 20 Tree Forest MSE: 0.5815170395339291


In [8]:
formula = " ~ " + " + ".join(features)
output  + formula

'Y ~ age_at_diagnosis + gender + JUN + VGLL3 + TERT + MAP3K5 + UST + CDKN2A + YAP1 + CDKN1B + PTPRQ + RB1 + TP53 + MYOCD + NF1 + CCNE1 + CEBPA + ZNF552 + ATRX + PTEN + DDIT3 + CDK4 + HMGA2 + MDM2 + FRS2 + Silent_per_Mb + Non_silent_per_Mb + CD274 + CTLA4 + HAVCR2 + LAG3 + PDCD1 + TCF7 + TIGIT'

In [39]:
# LMM

formula = " ~ " + " + ".join(features)

md = smf.mixedlm(output + formula, X_train, groups=X_train["short_histo"])
mdf = md.fit()
pred = mdf.predict(X_test)
lmm_mse = mse(X_test[output], pred)
print("LMM MSE:", lmm_mse)


LMM MSE: 0.3715772459434821


/opt/homebrew/anaconda3/envs/me_reg/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/anaconda3/envs/me_reg/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/opt/homebrew/anaconda3/envs/me_reg/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/anaconda3/envs/me_reg/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/opt/homebrew/anaconda3/envs/me_reg/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maxim

In [49]:
group_pred[10,:]

array([0.28199592, 0.01222614, 0.27322785, 0.18529095, 0.24725914])

# Visualizations

In [86]:
from sklearn.tree import plot_tree

fig, axs = plt.subplots(3, 2, figsize=(15, 20), dpi=800)




fig = plot_tree(list_of_trees[i], max_depth=2, filled=True, ax=axs[0, 0], feature_names=features)
axs[0, 0].set_title('STLMS')

fig = plot_tree(list_of_trees[i], max_depth=2, filled=True, ax=axs[0, 1], feature_names=features)
axs[0, 1].set_title('DDLPS')

fig = plot_tree(list_of_trees[i], max_depth=2, filled=True, ax=axs[1, 0], feature_names=features)
axs[1, 0].set_title('UPS')

fig = plot_tree(list_of_trees[i], max_depth=2, filled=True, ax=axs[1, 1], feature_names=features)
axs[1, 1].set_title('MFS')

fig = plot_tree(list_of_trees[i], max_depth=2, filled=True, ax=axs[2, 0], feature_names=features)
axs[2, 0].set_title('ULMS')

axs[2, 1].axis('off')

plt.savefig("sarcoma_tree.png")